# Now that we understand the indexing pipeline, we can 
# utilize our vector database to retrieve relevant documents
# for a given query and use them to generate a response.

In [1]:
%pip install -qU langchain langchain_community langchain_huggingface langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
# load the environment variables
%pip install -qU python-dotenv

from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

# lets just validate that we have the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

loader = WebBaseLoader("https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = InMemoryVectorStore.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
d:\Development\AI\retrievalAugmentedGen\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# setup the chat model and prompt template
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about the US Constitution. Use the provided context to answer the question. IMPORTANT: If you are unsure of the answer, say 'I don't know' and don't make up an answer."),
    ("user", "Question: {question}\nContext: {context}")
])

chain = prompt | llm

In [ ]:
query = "Is there any plans to change the US Constitution to allow for term limits on senators?"

# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs})

print(response.content)

I don't know. The provided context does not include any information on current plans or proposals to change the US Constitution to establish term limits for senators. If such a proposal exists, it would usually be discussed in legislative records or through public announcements by lawmakers.


In [9]:
while True:
    query = input("Enter a question: ")
    if query == "exit":
        break
    docs = retriever.invoke(query)
    response = chain.invoke({"question": query, "context": docs})
    print(response.content)


I don't know who the current president is based on the context provided. The documents you provided contain information about the powers, duties, and restrictions related to the presidency, but they do not specify who currently holds the office.
The President of the United States, as outlined in the context provided, is designated as the Commander in Chief of the Army and Navy of the United States, and of the Militia of the several States when they are called into actual service of the United States. This role gives the President significant authority over military operations and decisions, although they do not have the power to declare war, which is reserved for Congress.
The President has the power to grant reprieves and pardons for offenses against the United States, except in cases of impeachment. This means the President can forgive or lessen the punishment for federal crimes, but this power does not extend to impeachments.
In the context of the United States Constitution, "impeac